In [59]:
import numpy as np
from functools import lru_cache
import time
from numba import njit, prange

In [60]:
BOARD_SIZE = 12
BOARD_SIZE_1 = BOARD_SIZE - 1
BOARD_SIZE_3 = BOARD_SIZE - 3
BOARD_SIZE_4 = BOARD_SIZE - 4
BOARD_SIZE_5 = BOARD_SIZE - 5
NB_WIN = 4
MAX_DEPTH = 0
intermediate_boards = []

In [61]:
def human_play(board, symbole, last_played_move):
  while 1:
    col = int(input("colonne -> ")) - 1
    line = int(input("ligne -> ")) - 1
    if 0 <= col < BOARD_SIZE and 0 <= line < BOARD_SIZE and  board[line, col] == 0:
      break
  return col, line

In [62]:
def ai_play(depth):
  def method(board, symbole, last_played_move):
    start = time.time()
    global intermediate_boards
    if(np.all(board == 0)):
      middle = BOARD_SIZE // 2 - 1
      return middle, middle
    opp = 1 if symbole == 2 else 2
    board = np.copy(board)
    _, move = minimax(board, depth, -np.inf, np.inf, True, symbole, opp, last_played_move)
    print("Execution en", time.time() - start, "s")
    return move

  return method
  # for legal moves

In [63]:
@njit
def legal_moves(board, last_played_move):
  lines, cols = np.where(board != 0)
  col, line = last_played_move
  order = np.argsort(np.floor(np.sqrt((cols - col) ** 2 + (lines - line) ** 2)))
  lines = lines[order]
  cols = cols[order]
  done = []
  for line, col in zip(lines, cols):
    for i in prange(max(0, line - 1), min(BOARD_SIZE, line + 2)):
      for j in prange(max(0, col - 1), min(BOARD_SIZE, col + 2)):
        if (i, j) not in done and board[i, j] == 0:
          yield j, i
          done.append((i, j))

In [64]:
VALUES = (
    1, #1
    10, #2
    100, #3
    1000000, #4
    # 100000000, #5
    # 10000000000 #6
)

In [65]:
@njit
def heuristic(board, symbol):
  global BOARD_SIZE
  global BOARD_SIZE_1
  global BOARD_SIZE_4
  global BOARD_SIZE_5
  score = 0
  for i in prange(BOARD_SIZE):
    cOppLine, cPlayerLine, cOppCol, cPlayerCol = 0, 0, 0, 0
    cOppDiag, cPlayerDiag = [0, 0, 0, 0], [0, 0, 0, 0]
    for j in prange(BOARD_SIZE):
      # line
      tempScore, cOppLine, cPlayerLine = define_score(board[i, j], cOppLine, cPlayerLine, symbol)
      score += tempScore
      # col
      tempScore, cOppCol, cPlayerCol = define_score(board[j, i], cOppCol, cPlayerCol, symbol)
      score += tempScore

      # \ diag 
      # from trace to left [0]
      if j + i < BOARD_SIZE_4:
        tempScore, cOppDiag[0], cPlayerDiag[0] = define_score(board[j + i, j], cOppDiag[0], cPlayerDiag[0], symbol)
        score += tempScore
      # from trace + 1 to right [1]
      if j + i < BOARD_SIZE_5:
        tempScore, cOppDiag[1], cPlayerDiag[1] = define_score(board[j, j + i + 1], cOppDiag[1], cPlayerDiag[1], symbol)
        score += tempScore
      # / diag
      # from / to left [2]
      if BOARD_SIZE - i - j >= 1:
        tempScore, cOppDiag[2], cPlayerDiag[2] = define_score(board[BOARD_SIZE - 1 - j - i, j], cOppDiag[2], cPlayerDiag[2], symbol)
        score += tempScore
      # from / to right [3]
      if i + j < BOARD_SIZE_1:
        tempScore, cOppDiag[3], cPlayerDiag[3] = define_score(board[BOARD_SIZE - j - 1, j + i + 1], cOppDiag[3], cPlayerDiag[3], symbol)
        score += tempScore
  return score


In [66]:
@njit
def define_score(case, countOpp, countPlayer, symbol):
  global VALUES
  score = 0
  if case == 0:
    if countOpp != 0 or countPlayer != 0:
      score = VALUES[min(max(countOpp, countPlayer), 4) - 1] * (1 if countPlayer != 0 else -1)
      countOpp = countPlayer = 0
  elif case == symbol:
    countPlayer += 1
    if countOpp != 0:
      score = -VALUES[min(countOpp, 4) - 1]
      countOpp = 0
  else:
    countOpp += 1
    if countPlayer != 0:
      score = VALUES[min(4, countPlayer) - 1]
      countPlayer = 0
  return score, countOpp, countPlayer

In [67]:
@njit
def minimax(board, depth, alpha, beta, maximizingPlayer, curr, opp, last_played_move, max_depth = 0):
  if max_depth == 0:
    max_depth = depth
  
  symbol = curr if maximizingPlayer else opp
  if depth == 0:
    return heuristic(board, symbol) * (1 if maximizingPlayer else -1), None

  if maximizingPlayer:
    maxEval = -np.inf
    best_move = None
    for col, line in legal_moves(board, last_played_move):
      board[line, col] = symbol
      if np.sum(board == curr) > 3 and is_win(board, symbol):
          eval = 100000 / (max_depth - depth + 1) ** 2
      else:
        eval, _ = minimax(board, depth - 1, alpha, beta, False, curr, opp, (col, line), max_depth)
        eval /= (max_depth - depth + 1) ** 2
      if eval > maxEval:
        best_move = (col, line)
        maxEval = eval
      alpha = max(alpha, eval)
      board[line, col] = 0
      if eval == None:
        continue
      if beta <= alpha:
        break
    return maxEval, best_move

  else:
    minEval = np.inf
    best_move = None
    for col, line in legal_moves(board, last_played_move):
      board[line, col] = symbol
      if np.sum(board == curr) > 3 and is_win(board, opp):
          minEval = -1000000 / (max_depth - depth + 1) ** 2
      else:
        eval, _ = minimax(board, depth - 1, alpha, beta, True, curr, opp, (col, line), max_depth)
        eval /= (max_depth - depth + 1) ** 2
        if eval < 0:
          eval *= 10
      if eval < minEval:
        minEval = eval
        best_move = (col, line)
      beta = min(beta, eval)
      board[line, col] = 0
      if beta <= alpha:
        break
    return minEval, best_move


In [68]:
def initGame():
  return np.array([[0] * BOARD_SIZE for _ in range(BOARD_SIZE)])

In [69]:
@njit
def played_move_by_symbol(board, symbol):
  global BOARD_SIZE
  for i in prange(BOARD_SIZE):
    for j in prange(BOARD_SIZE):
      if board[i, j] == symbol:
        yield i, j

In [70]:
@njit
def is_win(board, symbol):
  global BOARD_SIZE
  global BOARD_SIZE_3
  for line, col in played_move_by_symbol(board, symbol):
      # line
      if line < BOARD_SIZE_3 and (board[line:line+4, col]==symbol).all():
          return True
      #col
      if col < BOARD_SIZE_3 and (board[line, col:col+4]==symbol).all():
          return True
      # \ diagonal
      if line < BOARD_SIZE_3 and col < BOARD_SIZE_3 and board[line, col] == board[line+1, col+1] == board[line+2, col+2] == board[line+3, col+3] == symbol:
          return True
      # / diagonal
      if col > 2 and line < BOARD_SIZE_3 and board[line, col] == board[line+1, col-1] == board[line+2, col-2] == board[line+3, col-3] == symbol:
          return True
      
  return False

In [71]:
def int_to_symbol(number):
  if number == 0:
    return ' '
  elif number == 1:
    return 'X'
  return 'O'

def print_board(board):
  first_line = '  ' + '   '.join(list(map(str, np.arange(12) + 1))) + '\n'
  line_sep = "\n  --+" + "---+" * max(0, BOARD_SIZE - 2) + "--\n"
  col_sep = list(map(lambda x : ' | '.join(list(map(int_to_symbol, x))), board))
  for i in range(len(col_sep)):
    col_sep[i] = str(i + 1) + (' ' if len(str(i + 1)) == 1 else '') + col_sep[i] 
  return first_line + line_sep.join(col_sep)

In [72]:
def start_game(player1, player2):
  gagnant = False
  symbole = 1
  num_moves = 0
  board = initGame()
  print(print_board(board))
  last_move = (6, 6)
  while not gagnant: # Tans que y'a pas de gagnant
      player = player1 if symbole == 1 else player2
      col, line = player(board, symbole, last_move)
      last_move = (col, line)
      board[line, col] = symbole
      print(print_board(board))
      print("Le joueur à joué:", (col + 1), (line + 1))
      num_moves += 1
      gagnant = is_win(board, symbole)
      if (not gagnant and num_moves == BOARD_SIZE ** 2) or gagnant:
          break
      symbole = 1 if symbole == 2 else 2
  print(print_board(board))
  print(int_to_symbol(symbole) + " won!")

In [82]:
start_game(human_play, ai_play(4))

  1   2   3   4   5   6   7   8   9   10   11   12
1   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
2   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
3   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
4   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
5   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
6   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
7   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
8   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
9   |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+---+---+--
10  |   |   |   |   |   |   |   |   |   |   |  
  --+---+---+---+---+---+---+---+---+